In [1]:
import json
import os
import matplotlib.pyplot as plt
import glob
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
import string, re
import tqdm 
import pandas as pd
import json
from transformers import AutoModelForCausalLM, AutoTokenizer

/Users/mariadobko/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
open_api_key = ""
secret_hf = ''

In [4]:
!huggingface-cli login --token $secret_hf

/Users/mariadobko/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/mariadobko/.cache/huggingface/token
Login successful


### Load data

In [3]:
data = json.load(open('pubmedqa-master/data/test_set.json'))
labels = json.load(open('pubmedqa-master/data/test_ground_truth.json'))
len(data), len(labels)

(500, 500)

In [1]:
predictions, gt = {}, {}

class_dict = { 'no': 0, 'yes': 1, 'maybe': 2, 'none':3}


model_id = "mistralai/Mixtral-8x7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)


for i, q in tqdm.tqdm(data.items()):
    question = str(q['QUESTION'])
    inp = {'question': question}
    text_prompt = "Answer question {}. Answer can only be one word and it should be either 'yes', 'no', or 'maybe'.".format(
        question)
    inputs = tokenizer(text_prompt, return_tensors="pt")

    outputs = model.generate(**inputs, max_new_tokens=20)
    resp = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(resp)
    break
    
    r = 'none'
    if 'no' in resp.lower():
        r = 'no'
    elif 'yes' in resp.lower():
        r = 'yes'
    elif 'maybe' in resp.lower() or 'perhaps' in resp.lower():
        r = 'maybe'
    
    predictions.update({i: class_dict[r]})
    gt.update({i: class_dict[labels[i]]})

In [14]:
len(predictions), len(gt)

(500, 500)

In [15]:
accuracy_score(list(gt.values()), list(predictions.values() ))

0.552

In [17]:
f1_score(list(gt.values()), list(predictions.values()), average='macro')

0.3875320857544917